The same as exp_1 but with LayerNorms and ReLU as activations

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../src')

In [3]:
import torch
from torch.utils.data import DataLoader

from flow_regressors.flow_dataset import FlowDataset
from flow_regressors.regressors import DensePoseRegressorV2

from training.train_flow import train
from training.loss_pose import LossPose

from tqdm.auto import tqdm

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Data

In [4]:
train_data = FlowDataset(
    root_path='/home/project/data_sample/flow/train',
    intrinsics_path='/home/project/ScanNet/scannet_indices/intrinsics.npz'
    )

train_loader = DataLoader(train_data, batch_size=128, shuffle=True, drop_last=True, pin_memory=True, num_workers=0)

val_data = FlowDataset(
    root_path='/home/project/data_sample/flow/val',
    intrinsics_path='/home/project/ScanNet/scannet_indices/intrinsics.npz'
    )

val_loader = DataLoader(val_data, batch_size=128, shuffle=False, drop_last=False, pin_memory=True, num_workers=0)

### 2. Configuration

In [5]:
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

config = dict(
    general = dict(
        experiment_name='FLOW_TO_POSE_EXP_5',
        device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'),
        
        n_epochs=10,
        n_steps_per_epoch=len(train_loader.dataset) // train_loader.batch_size,
        n_accum_steps=1,
        batch_size=train_loader.batch_size,
        
        swa=False,
        n_epochs_swa=None,
        n_steps_between_swa_updates=None,
          
        repeat_val_epoch=1,
        repeat_save_epoch=5,
          
        model_save_path='../../src/weights/FLOW_TO_POSE_EXP_5'
    )
)

### 3. Model

In [6]:
# Inference on (120, 160) input:

model = DensePoseRegressorV2(activation='relu')
model.to(config['general']['device']);

In [7]:
# data = train_data[0]
# data['flow'] = torch.from_numpy(data['flow']).cuda().unsqueeze(0) 
# data['K_q'] = torch.from_numpy(data['K_q']).cuda().unsqueeze(0) 
# data['K_s'] = torch.from_numpy(data['K_s']).cuda().unsqueeze(0) 
# model.eval();

In [8]:
# %%timeit

# with torch.no_grad():
#     q, t = model(
#         data['flow'], data['K_q'], data['K_s'], (0.25, 0.25), (0.25, 0.25)
#        )

### 4. Loss, optimizer, scheduler

In [9]:
train_loss = LossPose(agg_type='mean')
val_loss = LossPose()

In [10]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-6)
config['optimizer'] = optimizer.__dict__['defaults']

In [11]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                   step_size=2,
                   gamma=0.8)

### 6. Experiment

In [12]:
train(model, optimizer, scheduler, train_loss, val_loss, train_loader, val_loader, config, **config['general'])

wandb: Currently logged in as: kovanic. Use `wandb login --relogin` to force relogin


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 333/333 [04:55<00:00,  1.13it/s]


epoch 0: train loss=-8.808781580821181, val loss(q)=0.11174838527909281, val loss(t)=1.043799483553741


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 333/333 [04:56<00:00,  1.12it/s]


epoch 1: train loss=-10.165602281882164, val loss(q)=0.10093702865767551, val loss(t)=0.9745062930108597


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 333/333 [04:37<00:00,  1.20it/s]


epoch 8: train loss=-13.403956762641961, val loss(q)=0.0674335900382931, val loss(t)=0.4132441527516603


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 333/333 [04:39<00:00,  1.19it/s]

epoch 9: train loss=-13.619256037282693, val loss(q)=0.06516185837495289, val loss(t)=0.3937705582651076


Train batch loss,█▇▇▆▇▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁
Train loss epoch,█▆▅▄▃▃▂▂▁▁
Val loss epoch(q),█▆▄▄▃▃▂▂▁▁
Val loss epoch(t),█▇▅▄▃▂▂▁▁▁
Train batch loss,-13.52066
Train loss epoch,-13.61926
Val loss epoch(q),0.06516
Val loss epoch(t),0.39377
